In [90]:
%load_ext autoreload
%autoreload 2
import sys 
sys.path.insert(0, '../src/')
import pandas as pd 
from prophet import Prophet
import numpy as np 
import plotly.graph_objs as go

import utils 
import historical_average_topdown 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
sales_train_evaluaiton_root = pd.read_csv('../data/sales_train_evaluation_root.csv')
sales_train_validation_root = pd.read_csv('../data/sales_train_validation_root.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')

sales_train_validation = pd.read_csv('../data/sales_train_validation.csv')
sales_train_evaluaiton = pd.read_csv('../data/sales_train_evaluation.csv')

In [92]:
sample_submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
result_validation

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_evaluation,0.353268,0.332258,0.329658,0.331959,0.367534,0.432128,0.429450,0.358752,0.338066,0.335748,...,0.380156,0.444602,0.441676,0.370638,0.349527,0.346711,0.348686,0.383831,0.447902,0.444618
HOBBIES_1_002_CA_1_evaluation,0.299278,0.281479,0.279277,0.281226,0.311364,0.366086,0.363817,0.303924,0.286399,0.284436,...,0.322057,0.376654,0.374175,0.313994,0.296109,0.293724,0.295396,0.325170,0.379449,0.376667
HOBBIES_1_003_CA_1_evaluation,0.168126,0.158127,0.156890,0.157985,0.174915,0.205656,0.204382,0.170736,0.160891,0.159788,...,0.180922,0.211593,0.210201,0.176393,0.166346,0.165005,0.165945,0.182671,0.213164,0.211601
HOBBIES_1_004_CA_1_evaluation,1.967832,1.850797,1.836316,1.849133,2.047297,2.407110,2.392194,1.998379,1.883149,1.870242,...,2.117608,2.476595,2.460300,2.064590,1.946994,1.931309,1.942307,2.138078,2.494976,2.476685
HOBBIES_1_005_CA_1_evaluation,1.133962,1.066520,1.058176,1.065561,1.179753,1.387095,1.378499,1.151564,1.085163,1.077726,...,1.220270,1.427136,1.417746,1.189718,1.121954,1.112915,1.119253,1.232066,1.437728,1.427187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_evaluation,0.731574,0.688064,0.682680,0.687445,0.761116,0.894882,0.889337,0.742930,0.700091,0.695293,...,0.787255,0.920714,0.914656,0.767545,0.723827,0.717996,0.722084,0.794865,0.927548,0.920748
FOODS_3_824_WI_3_evaluation,0.492991,0.463671,0.460043,0.463254,0.512899,0.603041,0.599304,0.500644,0.471776,0.468542,...,0.530514,0.620449,0.616366,0.517231,0.487771,0.483841,0.486596,0.535642,0.625054,0.620471
FOODS_3_825_WI_3_evaluation,1.102940,1.037344,1.029227,1.036411,1.147479,1.349148,1.340788,1.120061,1.055476,1.048242,...,1.186887,1.388093,1.378960,1.157171,1.091260,1.082469,1.088633,1.198360,1.398396,1.388144


In [38]:
calender = pd.read_csv('../data/calendar.csv') 
date_to_d = dict(zip(calender.date, calender.d))

In [89]:
%%writefile ../src/historical_average_topdown.py

from prophet import Prophet 
import pandas as pd 
import numpy as np

def prophrt_pre_processing(df, calender): 


    df = df.rename({"Unnamed: 0":"day_index"})
    df.columns = ["day_index",'sales_quantity']

    df = df.merge(calender[['d','date']], right_on='d', left_on='day_index', how = 'left')

    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df = df[['date','sales_quantity']]
    df.columns = ['ds','y']

    return df 


def prophet_train(df, day_seasonality=True): 

    model = Prophet(daily_seasonality=day_seasonality )
    model.fit(df)

    return model 


def prophet_forecast(model,future): 

    future = model.make_future_dataframe(periods=future)
    forecasting = model.predict(future) 

    return forecasting 


def get_root_proportions(df , forecast_df, future ,name, sample_submission): 

    df = df.drop(columns = ['item_id','dept_id','cat_id','store_id','state_id'])
    df_values = df.values
    df_values = df_values[:,1:] 


    sum_by_day = df_values.sum(axis = 0)
    percentage_by_day = df_values/sum_by_day 

    historical_percentage = percentage_by_day.mean(axis = 1, keepdims=True ) 

    y_hat = forecast_df.tail(future)['yhat']
    y_hat = y_hat.values.reshape(-1,1) 

    leaf_forecast = np.dot(historical_percentage, np.transpose(y_hat))
    print(f"Leaft forecast matrix has shape:{leaf_forecast.shape}")

    result_float = pd.DataFrame(leaf_forecast, index= df['id'], columns = sample_submission.columns[1:] )
    result_float = result_float.astype(float)
    
    return result_float 


def post_processing(valid, eval, sample_submission, INTRESULT = False): 

    result = pd.concat((valid, eval), axis = 0)

    if INTRESULT:
        result = result.apply(np.round) 

    result = result.reset_index() 

    # validation
    for result_id, submission_id in zip(list(result['id']), list(sample_submission['id'])): 

        if result_id != submission_id: 
            print(result_id, submission_id)
            raise "Incorrect submmission index found"

    return result 



Overwriting ../src/historical_average_topdown.py


In [97]:
df = sales_train_validation_root 
future  = 28 
df_processed = historical_average_topdown.prophrt_pre_processing(df,calender) 
model = historical_average_topdown.prophet_train(df_processed, day_seasonality = True)
forecast = historical_average_topdown.prophet_forecast(model, 28)
result_validation = historical_average_topdown.get_root_proportions(sales_train_validation, forecast, future ,'sales_train_validation', sample_submission)
result_validation.head()

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -27.3885
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        4390.2   0.000462918       88.5894      0.3819      0.3819      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     187       4393.31   7.18846e-05       123.548    4.57e-07       0.001      287  LS failed, Hessian reset 
     199       4393.44    0.00030333       97.6894           1           1      303   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     245       4393.83   0.000638502       324.915   8.284e-06       0.001      411  LS failed, Hessian reset 
     299       4394.17   7.02267e-05       51.0474           1           1      481   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     317        4394.3   0.000949357         209.5   1.318e-05       0.001      576  LS failed, Hessian rese

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Leaft forecast matrix has shape:(30490, 28)


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0.330914,0.309658,0.306244,0.307492,0.340842,0.402224,0.398394,0.328874,0.307748,0.304468,...,0.339529,0.401431,0.398173,0.329277,0.308828,0.306280,0.308451,0.342781,0.405199,0.402457
HOBBIES_1_002_CA_1_validation,0.289849,0.271231,0.268241,0.269334,0.298545,0.352310,0.348956,0.288063,0.269558,0.266685,...,0.297395,0.351616,0.348762,0.288415,0.270504,0.268272,0.270174,0.300244,0.354916,0.352514
HOBBIES_1_003_CA_1_validation,0.154946,0.144994,0.143395,0.143980,0.159595,0.188337,0.186543,0.153991,0.144099,0.142564,...,0.158980,0.187966,0.186440,0.154180,0.144605,0.143412,0.144429,0.160503,0.189730,0.188446
HOBBIES_1_004_CA_1_validation,1.904414,1.782086,1.762438,1.769622,1.961548,2.314804,2.292765,1.892675,1.771095,1.752218,...,1.953995,2.310244,2.291491,1.894992,1.777313,1.762644,1.775140,1.972710,2.331926,2.316144
HOBBIES_1_005_CA_1_validation,1.091443,1.021336,1.010075,1.014192,1.124188,1.326643,1.314012,1.084716,1.015037,1.004218,...,1.119859,1.324030,1.313282,1.086043,1.018600,1.010193,1.017355,1.130585,1.336456,1.327411


In [99]:
df = sales_train_evaluaiton_root
future  = 28 
df_processed = historical_average_topdown.prophrt_pre_processing(df,calender) 
model = historical_average_topdown.prophet_train(df_processed, day_seasonality = True)
forecast = historical_average_topdown.prophet_forecast(model, 28)
result_evaluation = historical_average_topdown.get_root_proportions(sales_train_evaluaiton, forecast, future ,'sales_train_validation', sample_submission)
result_evaluation.head()

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -35.7339
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4539.14    0.00493046       173.115           1           1      135   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       4542.53    0.00186406       172.413           1           1      257   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     225       4542.89   9.04218e-05       168.862   3.941e-07       0.001      332  LS failed, Hessian reset 
     299       4543.25    0.00163571       149.019           1           1      433   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     380       4543.53   4.88214e-07       43.8024      0.5666      0.5666      551   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Leaft forecast matrix has shape:(30490, 28)


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_evaluation,0.353268,0.332258,0.329658,0.331959,0.367534,0.432128,0.429450,0.358752,0.338066,0.335748,...,0.380156,0.444602,0.441676,0.370638,0.349527,0.346711,0.348686,0.383831,0.447902,0.444618
HOBBIES_1_002_CA_1_evaluation,0.299278,0.281479,0.279277,0.281226,0.311364,0.366086,0.363817,0.303924,0.286399,0.284436,...,0.322057,0.376654,0.374175,0.313994,0.296109,0.293724,0.295396,0.325170,0.379449,0.376667
HOBBIES_1_003_CA_1_evaluation,0.168126,0.158127,0.156890,0.157985,0.174915,0.205656,0.204382,0.170736,0.160891,0.159788,...,0.180922,0.211593,0.210201,0.176393,0.166346,0.165005,0.165945,0.182671,0.213164,0.211601
HOBBIES_1_004_CA_1_evaluation,1.967832,1.850797,1.836316,1.849133,2.047297,2.407110,2.392194,1.998379,1.883149,1.870242,...,2.117608,2.476595,2.460300,2.064590,1.946994,1.931309,1.942307,2.138078,2.494976,2.476685
HOBBIES_1_005_CA_1_evaluation,1.133962,1.066520,1.058176,1.065561,1.179753,1.387095,1.378499,1.151564,1.085163,1.077726,...,1.220270,1.427136,1.417746,1.189718,1.121954,1.112915,1.119253,1.232066,1.437728,1.427187


In [102]:
result = historical_average_topdown.post_processing(result_validation, result_evaluation, sample_submission,)
result.head()
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60980 entries, 0 to 60979
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      60980 non-null  object 
 1   F1      60980 non-null  float64
 2   F2      60980 non-null  float64
 3   F3      60980 non-null  float64
 4   F4      60980 non-null  float64
 5   F5      60980 non-null  float64
 6   F6      60980 non-null  float64
 7   F7      60980 non-null  float64
 8   F8      60980 non-null  float64
 9   F9      60980 non-null  float64
 10  F10     60980 non-null  float64
 11  F11     60980 non-null  float64
 12  F12     60980 non-null  float64
 13  F13     60980 non-null  float64
 14  F14     60980 non-null  float64
 15  F15     60980 non-null  float64
 16  F16     60980 non-null  float64
 17  F17     60980 non-null  float64
 18  F18     60980 non-null  float64
 19  F19     60980 non-null  float64
 20  F20     60980 non-null  float64
 21  F21     60980 non-null  float64
 22

In [106]:
result.to_csv('../data/submissions/prophet_root_simple_average_proportions.csv', index=False)

In [61]:
def plot_forecast(x, y_hat, y_hat_lower, y_hat_upper, y):

    x = x.apply(lambda x: x.strftime('%Y-%m-%d'))

    fig = go.Figure([
        go.Scatter(
            name='y_hat',
            x=x,
            y=y_hat,
            mode='lines',
            line=dict(color='rgb(31, 119, 180)'),
        ),
        go.Scatter(
            name='Upper Bound',
            x=x,
            y=y_hat_upper,
            mode='lines',
            marker=dict(color="#444"),
            line=dict(width=0),
            showlegend=False
        ),
        go.Scatter(
            name='Lower Bound',
            x=x,
            y=y_hat_lower,
            marker=dict(color="#444"),
            line=dict(width=0),
            mode='lines',
            fillcolor='rgba(68, 68, 68, 0.3)',
            fill='tonexty',
            showlegend=False
        ),
        go.Scatter(
            name='y',
            x=x,
            y=y,
            mode='lines',
            line=dict(color='rgb(25, 190, 70)'),
        ),
    ])

    fig.update_layout(
        yaxis_title='Forecasted Sales Quantity',
        title='Comparison',
        hovermode="x"
    )

    
    fig.show()

In [62]:
plot_forecast(x, y_hat, y_hat_lower, y_hat_upper,y)